In [102]:
import numpy as np
from StringIO import StringIO
import string
from scipy import stats
from sklearn import datasets, linear_model
import math
import matplotlib.pyplot as plt

In [103]:
# this imports our data
data = np.genfromtxt('movie_metadata.csv', dtype=None, delimiter=',', autostrip=True)

# this is the uncropped data set
fullDataSet = data

# this chops out the director name, actor 2 name, actor 1 name, movie Title, actor 3 name, plot keywords,
# movie imbd link, language, country, content rating
data = np.delete(data, [2, 7, 11, 12, 15, 17, 18, 20, 21, 22], 1)

# reducing "data" matrix to only numeric data 
onlyNumericData  = np.delete(data, [1,10,12,13], 1)

#vector containing feature names 
colNames = data[0,:]

# cuts out feature label row
movieData = np.delete(data, (0), axis=0)

# finds length,numMovies, and width,numFeatures, of dataset and prints
numMovies = movieData.shape[0]
numFeatures = movieData.shape[1]
print numMovies 
print numFeatures

3756
19


In [104]:
# STARTS EASYDEX FUNCTION AND EXAMPLES

In [105]:
# This function provides an easy way of indexing our data set. 

#The arguments are the movieID, the feature you want or 0 for all features , followed by a boolean 1 or 0 for 
# the cropped data set "data" or the uncropped data set "fullDataSet"

#
# example function calls are: easydex(112,'movieID',1)  
#         This returns the 112 movie's movieID which is 112. 
# example function calls are: easydex(460,0,1)
#         This gives all the data on the 460th movie. 
# example function calls are: easydex(460,0,0)
#         This gives all the data on the 460th movie from the "fullDataSet" 
#         instead of the cropped "data" matrix.

def easydex(MovieRow, FeatureString, selectDataSet):
    
    if selectDataSet == 1:
        dataSet = data
    elif selectDataSet == 2:
        dataSet = onlyNumericData 
    else: 
        dataSet = fullDataSet
    
    MovieRow = MovieRow -1
    my_data_dict= {}
   
    col_names = dataSet[0,:]
   
    # returns specific feature for a specific movie
    if FeatureString != 0:
        for l in range(0,numFeatures):
            my_data_dict[col_names[l]]= dataSet[1:,l]
   
        example1 = my_data_dict[FeatureString]
        return[example1[MovieRow]]
    
    #returns whole row for a specific movie
    else:
        my_data_neat_index = dataSet[1:,:]
        myMovieSample = my_data_neat_index[MovieRow,:]
        return myMovieSample

In [106]:
#testing easydex function, output should be 112
easydex(112,'movieID',1)

['117']

In [107]:
#testing easydex function output
#This gives all the data on the 1st movie from the "fullDataSet" 
#instead of the cropped "data" matrix.
easydex(1,0,0)



array(['1', 'Color', 'James Cameron', '723', '178', '0', '855',
       'Joel David Moore', '1000', '760505847',
       'Action|Adventure|Fantasy|Sci-Fi', 'CCH Pounder', '"""Avatar"""',
       '886204', '4834', 'Wes Studi', '0',
       'avatar|future|marine|native|paraplegic',
       'http://www.imdb.com/title/tt0499549/?ref_=fn_tt_tt_1', '3054',
       'English', 'USA', 'PG-13', '237000000', '2009', '936', '7.9',
       '1.78', '33000'], 
      dtype='|S149')

In [108]:
#testing easydex function output should be all data in "data" matrix on movie 1
easydex(1,0,1)

array(['1', 'Color', '723', '178', '0', '855', '1000', '760505847',
       'Action|Adventure|Fantasy|Sci-Fi', '886204', '4834', '0', '3054',
       '237000000', '2009', '936', '7.9', '1.78', '33000'], 
      dtype='|S149')

In [109]:
#testing easydex function output
#This gives all the data on the 460th movie from the "fullDataSet" 
#instead of the cropped "data" matrix.
easydex(1,0,2)

array(['1', '723', '178', '0', '855', '1000', '760505847',
       'Action|Adventure|Fantasy|Sci-Fi', '886204', '0', '2009', '936',
       '7.9', '1.78', '33000'], 
      dtype='|S149')

In [110]:
# ENDS EASYDEX FUNCTION AND EXAMPLES

In [111]:
onlyNumericData = onlyNumericData[1:,:]

In [112]:
# Normalize the data 
#stats.zscore(movieData)

print (easydex(1,0,2))
print(onlyNumericData[1,:])

#onlyNumericData = onlyNumericData.astype(float)


onlyNumericData = onlyNumericData.astype(np.float)
print(onlyNumericData[1,:])

#stats.zscore(onlyNumericData , axis=2, ddof=1)

['2' '302' '169' '563' '1000' '40000' '309404152'
 'Action|Adventure|Fantasy' '471220' '0' '2007' '5000' '7.1' '2.35' '0']
['2' '302' '169' '563' '1000' '40000' '309404152'
 'Action|Adventure|Fantasy' '471220' '0' '2007' '5000' '7.1' '2.35' '0']


ValueError: could not convert string to float: Action|Adventure|Fantasy|Sci-Fi

In [113]:
genres = {'Action', 'Adventure','Animation','Biography','Comedy','Crime','Documentary','Drama','Family','Fantasy',
          'Film-Noir','Game-Show','History','Horror','Music','Musical','Mystery','News','Reality-TV','Romance','Sci-Fi',
          'Short','Sport','Thriller','War','Western'}

moviesByGenreBucket = {}

for genre in genres:
    moviesByGenreBucket[genre] = []

In [114]:
# buckets movieIDs into genre buckets
for movie in movieData:    
    localGenres = string.split(movie[8], "|")
    for genre in localGenres:
        #print "adding movieID = %s to genre bucket %s" % (movie[0], genre)
        moviesByGenreBucket[genre].append(movie[0])
    
    # one hot coding for color feature
    if movie[1] == 'Color':
        movie[1] = 1
    else:
        movie[1] = 0

# remove genre column now that we have consumed it
movieData = np.delete(movieData, 8, 1)

# cast dytpe to float
movieData = movieData.astype(np.float)

In [137]:
# Zscore my data
stats.zscore(movieData, axis=0)

array([[-1.59022935,  0.18477275,  4.50130796, ...,  1.35849773,
        -0.93766206,  1.10187025],
       [-1.589486  ,  0.18477275,  1.09062269, ...,  0.6009981 ,
         0.67697286, -0.43587211],
       [-1.58874264,  0.18477275,  3.52103975, ...,  0.31693574,
         0.67697286,  3.52497943],
       ..., 
       [ 2.14958324,  0.18477275, -0.19749835, ...,  0.50631065,
        -0.73937356,  0.44949471],
       [ 2.15106994,  0.18477275, -0.90231929, ...,  0.4116232 ,
        -2.09906612, -0.43587211],
       [ 2.15627342,  0.18477275, -1.00763737, ...,  0.12756084,
        -0.73937356, -0.41462331]])

In [138]:
# Split data into Test, Train
dataSplitRate = 0.7
numSamples = int(math.floor(numMovies * dataSplitRate))
train = movieData[:numSamples].astype(np.float, casting='unsafe')
test = movieData[numSamples:].astype(np.float, casting='unsafe')
trainX = np.delete(train, 15, axis=1)
trainY = train[:,15]
testX = np.delete(test, 15, axis=1)
testY = test[:,15]

In [142]:
# linear regression
regr = linear_model.LinearRegression()
regr.fit(trainX, trainY)

# The coefficients
print('Coefficients: \n', regr.coef_)

# The mean squared error
print("Mean squared error: %.2f"
      % np.mean((regr.predict(testX) - testY) ** 2))

# The mean squared error
print("Median squared error: %.2f"
      % np.median((regr.predict(testX) - testY) ** 2))

# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % regr.score(testX, testY))



('Coefficients: \n', array([  1.96244502e-04,  -3.55678194e-01,   2.80247484e-03,
         1.27207684e-02,   6.77772711e-06,   5.03001066e-05,
         5.74480490e-05,   1.98991180e-10,   3.49006248e-06,
        -5.43981456e-05,  -2.98439219e-02,  -6.62235361e-04,
         8.71256311e-11,  -1.83375701e-02,   5.98358426e-05,
         3.76579688e-02,  -3.49619865e-06]))
Mean squared error: 0.79
Median squared error: 0.30
Variance score: 0.30
